In [7]:
import cv2
import numpy as np
import tensorflow as tf
import os

# Load the model without compiling
loaded_model = tf.keras.models.load_model(r"C:\Users\nabee\OneDrive\Desktop\deep learning\Lab4&5\best_model.h5", compile=False)

# Path to the folder containing class names
class_names_folder = r"C:\Users\nabee\Downloads\Lab 5\Lab 5\105_classes_pins_dataset"
folder_names = sorted(os.listdir(class_names_folder))

# Initialize class label map with "not identified"
class_label_map = {0: "not identified"}
for label, folder_name in enumerate(folder_names, 1):
    class_name = folder_name.replace("pins_", "")
    class_label_map[label] = class_name

# Function to compute cosine similarity
def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Generate reference embeddings for each class
reference_embeddings = {}
for label, class_name in class_label_map.items():
    if class_name == "not identified":
        continue  # Skip "not identified" label
    class_folder = os.path.join(class_names_folder, f"pins_{class_name}")
    sample_image_path = os.path.join(class_folder, os.listdir(class_folder)[0])  # Use the first image in the folder
    image = cv2.imread(sample_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_image = cv2.resize(image_rgb, (160, 160)).astype('float32') / 255.0
    input_image = np.expand_dims(input_image, axis=0)
    embedding = loaded_model.predict(input_image)
    reference_embeddings[label] = embedding.flatten()  # Store the flattened embedding

# Load OpenCV face detector
pb_files_path = r"C:\Users\nabee\Downloads\Lab 5\Lab 5\opencv_face_detector_uint8.pb"
pbtxt_files_path = r"C:\Users\nabee\Downloads\Lab 5\Lab 5\opencv_face_detector.pbtxt"
net = cv2.dnn.readNetFromTensorflow(pb_files_path, pbtxt_files_path)

# Face detection function
def detect_faces(net, frame):
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104., 177., 123.], False, False)
    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 3)
    return frame

# Start capturing video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_with_rectangles = detect_faces(net, frame)

    # Convert to RGB and preprocess
    frame_rgb = cv2.cvtColor(frame_with_rectangles, cv2.COLOR_BGR2RGB)
    input_frame = cv2.resize(frame_rgb, (160, 160)).astype('float32') / 255.0
    input_frame = np.expand_dims(input_frame, axis=0)

    # Generate embedding for the detected face
    embedding = loaded_model.predict(input_frame).flatten()

    # Calculate cosine similarity with reference embeddings
    max_similarity = -1
    predicted_class_index = 0  # Default to "not identified"
    for label, ref_embedding in reference_embeddings.items():
        similarity = cosine_similarity(embedding, ref_embedding)
        if similarity > max_similarity:
            max_similarity = similarity
            predicted_class_index = label

    # Threshold for "not identified"
    if max_similarity < 0.6:
        predicted_class_index = 0  # "not identified"

    # Display the result
    predicted_class_name = class_label_map[predicted_class_index]
    cv2.putText(frame_with_rectangles, f'Prediction: {predicted_class_name}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Live Stream', frame_with_rectangles)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━